In [ ]:
import numpy as np
import pandas as pd

import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from keras.utils import np_utils

import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('/content/train_emoji.csv',header=None)
test = pd.read_csv('/content/test_emoji.csv',header=None)
train.drop(labels=[2,3], axis= 1, inplace = True )

In [ ]:
train.head()



,0,1
0,never talk to me again,3
1,I am proud of your achievements,2
2,It is the worst day in my life,3
3,Miss you so much,0
4,food is life,4


In [ ]:
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [ ]:
!pip install emoji

     |████████████████████████████████| 184 kB 7.8 MB/s 
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=dee370849164e3ff18680bb5c975847497eedf2790e11cb08ab7f33b3d1f8125
  Stored in directory: /root/.cache/pip/wheels/e4/61/e7/2fc1ac8f306848fc66c6c013ab511f0a39ef4b1825b11363b2
Successfully built emoji


In [ ]:
emoji_dict = { 0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:"}

In [ ]:
for ix in emoji_dict.keys():
    print (ix,end=" ")
    print (emoji.emojize(emoji_dict[ix], use_aliases=True))

0 ❤
1 ⚾
2 😄
3 😞
4 🍴


In [ ]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print (X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
print ("-------------------------")
print (X_train[0],Y_train[0])

(132,) (132,) (56,) (56,)
-------------------------
never talk to me again 3


In [ ]:
# Splitting the train data from sentences to words
for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()

# Splitting the test data from sentences to words
for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()
    
# Converting labels into categorical form
Y_train = np_utils.to_categorical(Y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
print (X_train[0],Y_train[0])

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [ ]:
np.unique(np.array([len(ix) for ix in X_train]) , return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 4,  5, 26, 35, 20, 21, 11,  5,  1,  4]))

In [ ]:
np.unique(np.array([len(ix) for ix in X_test]) , return_counts=True)

(array([2, 3, 4, 5, 6, 7, 8]), array([ 3, 12, 16, 17,  3,  4,  1]))

In [ ]:
embeddings_index = {}

f = open("/content/glove.6B.50d.txt", encoding='utf8' , mode = 'r')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
embeddings_index["i"].shape

(50,)

In [ ]:
from scipy import spatial
# Checking cosine similarity of words happy and sad
spatial.distance.cosine(embeddings_index["happy"], embeddings_index["sad"])

0.31093674898147583

In [ ]:
spatial.distance.cosine(embeddings_index["india"], embeddings_index["delhi"])

0.18572336435317993

In [ ]:
spatial.distance.cosine(embeddings_index["france"], embeddings_index["paris"])

0.19746702909469604

In [ ]:
# Filling the embedding matrix
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embedding_matrix_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
        
for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        embedding_matrix_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]

In [ ]:
print (embedding_matrix_train.shape, embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


In [ ]:

model = Sequential()
model.add(SimpleRNN(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 10, 64)            7360      
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 15,941
Trainable params: 15,941
Non-trainable params: 0
____________________________________________________

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
5/5 [==============================] - 22s 13ms/step - loss: 1.6458 - accuracy: 0.1794
Epoch 2/50
5/5 [==============================] - 0s 7ms/step - loss: 1.6030 - accuracy: 0.2816
Epoch 3/50
5/5 [==============================] - 0s 6ms/step - loss: 1.5511 - accuracy: 0.3179
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - loss: 1.5936 - accuracy: 0.2173
Epoch 5/50
5/5 [==============================] - 0s 7ms/step - loss: 1.5408 - accuracy: 0.3525
Epoch 6/50
5/5 [==============================] - 0s 6ms/step - loss: 1.5010 - accuracy: 0.3476
Epoch 7/50
5/5 [==============================] - 0s 6ms/step - loss: 1.4766 - accuracy: 0.3194
Epoch 8/50
5/5 [==============================] - 0s 6ms/step - loss: 1.4083 - accuracy: 0.3518
Epoch 9/50
5/5 [==============================] - 0s 8ms/step - loss: 1.4385 - accuracy: 0.3365
Epoch 10/50
5/5 [==============================] - 0s 6ms/step - loss: 1.4700 - accuracy: 0.3525
Epoch 11/50
5/5 [====================

In [ ]:
pred = model.predict_classes(embedding_matrix_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.32142857142857145

In [ ]:
for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print(ix)
        print (test[0][ix],end=" ")
        print (emoji.emojize(emoji_dict[pred[ix]], use_aliases=True),end=" ")
        print (emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True))

0
['I', 'want', 'to', 'eat'] 😄 🍴
1
['he', 'did', 'not', 'answer'] 😄 😞
3
['she', 'got', 'me', 'a', 'present'] 😄 ❤
5
['he', 'is', 'a', 'good', 'friend'] 😄 ❤
6
['I', 'am', 'upset'] 😄 ❤
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 😄 ❤
8
['where', 'is', 'the', 'food'] 😄 🍴
10
['where', 'is', 'the', 'ball'] 😄 ⚾
11
['work', 'is', 'hard'] 😄 😞
12
['This', 'girl', 'is', 'messing', 'with', 'me'] 😄 😞
14
['Let', 'us', 'go', 'play', 'baseball'] 😄 ⚾
15
['This', 'stupid', 'grader', 'is', 'not', 'working'] 😄 😞
16
['work', 'is', 'horrible'] 😄 😞
18
['stop', 'messing', 'around'] 😄 😞
19
['any', 'suggestions', 'for', 'dinner'] 😄 🍴
20
['I', 'love', 'taking', 'breaks'] 😄 ❤
22
['I', 'boiled', 'rice'] 😄 🍴
23
['she', 'is', 'a', 'bully'] 😄 😞
24
['Why', 'are', 'you', 'feeling', 'bad'] 😄 😞
25
['I', 'am', 'upset'] 😄 😞
26
['I', 'worked', 'during', 'my', 'birthday'] 😄 😞
27
['My', 'grandmother', 'is', 'the', 'love', 'of', 'my', 'life'] 😄 ❤
29
['valentine', 'day', 'is', 'near'] 😄 ❤
30
['I', 'miss', 'you', 

In [ ]:
x = ['i', 'do', 'think','this', 'class', 'is', 'very', 'interesting']

x_ = np.zeros((1,10,50))

for ix in range(len(x)):
    x_[0][ix] = embeddings_index[x[ix].lower()]

In [ ]:
model.predict_classes(x_)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([3])

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 128)           91648     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 223,877
Trainable params: 223,877
Non-trainable params: 0
________________________________________________

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
5/5 [==============================] - 5s 24ms/step - loss: 1.5987 - accuracy: 0.2657
Epoch 2/50
5/5 [==============================] - 0s 24ms/step - loss: 1.5522 - accuracy: 0.3377
Epoch 3/50
5/5 [==============================] - 0s 25ms/step - loss: 1.5464 - accuracy: 0.3031
Epoch 4/50
5/5 [==============================] - 0s 23ms/step - loss: 1.5328 - accuracy: 0.2701
Epoch 5/50
5/5 [==============================] - 0s 24ms/step - loss: 1.5258 - accuracy: 0.3127
Epoch 6/50
5/5 [==============================] - 0s 22ms/step - loss: 1.4856 - accuracy: 0.3314
Epoch 7/50
5/5 [==============================] - 0s 21ms/step - loss: 1.4950 - accuracy: 0.3339
Epoch 8/50
5/5 [==============================] - 0s 22ms/step - loss: 1.4479 - accuracy: 0.3402
Epoch 9/50
5/5 [==============================] - 0s 24ms/step - loss: 1.4414 - accuracy: 0.3456
Epoch 10/50
5/5 [==============================] - 0s 22ms/step - loss: 1.4562 - accuracy: 0.3531
Epoch 11/50
5/5 [============

In [ ]:
pred = model.predict_classes(embedding_matrix_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.26785714285714285

In [ ]:
for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print(ix)
        print (test[0][ix],end=" ")
        print (emoji.emojize(emoji_dict[pred[ix]], use_aliases=True),end=" ")
        print (emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True))

0
['I', 'want', 'to', 'eat'] 😞 🍴
2
['he', 'got', 'a', 'raise'] 😞 😄
3
['she', 'got', 'me', 'a', 'present'] 😞 ❤
4
['ha', 'ha', 'ha', 'it', 'was', 'so', 'funny'] 😞 😄
5
['he', 'is', 'a', 'good', 'friend'] 😞 ❤
6
['I', 'am', 'upset'] 😞 ❤
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 😞 ❤
8
['where', 'is', 'the', 'food'] 😞 🍴
9
['Stop', 'making', 'this', 'joke', 'ha', 'ha', 'ha'] 😞 😄
10
['where', 'is', 'the', 'ball'] 😞 ⚾
13
['are', 'you', 'serious', 'ha', 'ha'] 😞 😄
14
['Let', 'us', 'go', 'play', 'baseball'] 😞 ⚾
17
['Congratulation', 'for', 'having', 'a', 'baby'] 😞 😄
19
['any', 'suggestions', 'for', 'dinner'] 😞 🍴
20
['I', 'love', 'taking', 'breaks'] 😞 ❤
21
['you', 'brighten', 'my', 'day'] 😞 😄
22
['I', 'boiled', 'rice'] 😞 🍴
27
['My', 'grandmother', 'is', 'the', 'love', 'of', 'my', 'life'] 😞 ❤
28
['enjoy', 'your', 'break'] 😞 😄
29
['valentine', 'day', 'is', 'near'] 😞 ❤
30
['I', 'miss', 'you', 'so', 'much'] 😞 ❤
31
['throw', 'the', 'ball'] 😞 ⚾
33
['she', 'said', 'yes'] 😞 😄
34
['will', '